In [1]:
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])

In [2]:
%%time
#th to zh
translator.translate("ฉันไปกินข้าว", src='th', dest='zh-cn').text

Wall time: 212 ms


'我去吃东西了'

In [3]:
%%time
#zh to th
translator.translate("我去吃饭", src='zh-cn',dest='th').text

Wall time: 235 ms


'ฉันกำลังจะไปกิน'

In [4]:
%%time
#zh to th
translator.translate("Have a good day people", src='en',dest='zh-cn').text

Wall time: 257 ms


'祝大家有个美好的一天'

In [11]:
translator.translate("Have a good day people", src='en',dest='zh-cn')

In [113]:
import pandas as pd

In [114]:
df = pd.read_csv("../data/en-th.merged_stratified.train.csv", encoding='utf-8')

In [127]:
%%time
from tqdm.auto import tqdm
x = []
for i in tqdm(df.en.tolist()):
    x.append(translator.translate(i, src='en',dest='zh-cn').text)
    time.sleep(random.randint(1,5))

ConnectTimeout: timed out